In [22]:
import pandas as pd
import numpy as np
name = "billing_and_accruals_for_script_2024-06-03_08-46-01"

In [23]:
dtype_dict = {
    'КПП': str,
}

In [24]:
dataset = pd.read_excel('C:/Users/Boris/Downloads/' + name + '.xlsx', dtype=dtype_dict)

In [25]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8293 entries, 0 to 8292
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Название компании           8202 non-null   object        
 1   Дата                        8293 non-null   datetime64[ns]
 2   Личный кабинет              8293 non-null   int64         
 3   ЛС                          8134 non-null   object        
 4   ИНН                         7323 non-null   object        
 5   КПП                         4773 non-null   object        
 6   Номер договора              8134 non-null   object        
 7   Идентификатор номенклатуры  8134 non-null   float64       
 8   Название номенклатуры       8134 non-null   object        
 9   Стоимость                   8293 non-null   float64       
 10  Сумма                       8293 non-null   float64       
 11  Начисления partner          4403 non-null   float64     

In [26]:
dataset['КПП'] = dataset['КПП'].astype(str)

In [27]:
dataset.head(2)

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,NaN
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,NaN


In [28]:
dataset['Начисления partner'] = dataset['Начисления partner'].fillna(0)

In [29]:
dataset['Сумма-test'] = dataset['Сумма']

In [30]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8288,ООО Связь-КТВ,2024-05-31 03:00:00,151195,Подписка ПРО ЛК 151195,NaN,nan,Подписка ПРО ЛК 151195,40.0,Подписка ПРО,2.7,35.1,10.0,35.1
8289,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-04-30 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,2.7,80.0,2.7
8290,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-05-31 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,91.8,0.0,91.8
8291,ООО Комфорт,2024-05-31 03:00:00,151399,Подписка ПРО ЛК 151399,5904130450,nan,Подписка ПРО ЛК 151399,40.0,Подписка ПРО,2.7,5.4,0.0,5.4


In [31]:
for i in range(0,dataset.shape[0]):
    if i == 0:
        date = dataset.loc[0,'Дата']
        lk = dataset.loc[0,'Личный кабинет'] 
        dataset.loc[0,'ballance'] = 0
        dataset.loc[0,'ballance'] = dataset.loc[0,'Сумма'] - dataset.loc[0, 'Начисления partner']
        if dataset.loc[0,'ballance'] < 0:
            dataset.loc[0,'Сумма']  = 0
        if dataset.loc[0,'ballance'] >= 0:
                dataset.loc[0,'Сумма'] = dataset.loc[0,'ballance'] 
                dataset.loc[0,'ballance'] = 0
            
    else:
    
        if date == dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if date != dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] - dataset.loc[i, 'Начисления partner'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if lk != dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            lk = dataset.loc[i,'Личный кабинет']
            
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] \
                                        - dataset.loc[i, 'Начисления partner']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0



In [32]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test,ballance
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0,0.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0,0.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8288,ООО Связь-КТВ,2024-05-31 03:00:00,151195,Подписка ПРО ЛК 151195,NaN,nan,Подписка ПРО ЛК 151195,40.0,Подписка ПРО,2.7,25.1,10.0,35.1,0.0
8289,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-04-30 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,0.0,80.0,2.7,-77.3
8290,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-05-31 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,14.5,0.0,91.8,0.0
8291,ООО Комфорт,2024-05-31 03:00:00,151399,Подписка ПРО ЛК 151399,5904130450,nan,Подписка ПРО ЛК 151399,40.0,Подписка ПРО,2.7,5.4,0.0,5.4,0.0


In [33]:
dataset.to_excel('C:/Users/Boris/Downloads/'+ name + '_test.xlsx', index=False)

In [34]:
dataset_ready = dataset.drop(['Начисления partner','Сумма-test','ballance'], axis= 1)

In [35]:
dataset_ready['Количество'] = round(dataset_ready['Сумма']/dataset_ready['Стоимость'])
dataset_ready['Количество'] = dataset_ready['Количество'].fillna(0) 

In [36]:
dataset_ready

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Количество
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,29.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,23.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
8288,ООО Связь-КТВ,2024-05-31 03:00:00,151195,Подписка ПРО ЛК 151195,NaN,nan,Подписка ПРО ЛК 151195,40.0,Подписка ПРО,2.7,25.1,9.0
8289,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-04-30 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,0.0,0.0
8290,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ЭЛЕВ-М""",2024-05-31 03:00:00,151221,Подписка ПРО ЛК 151221,7743429025,774301001,Подписка ПРО ЛК 151221,40.0,Подписка ПРО,2.7,14.5,5.0
8291,ООО Комфорт,2024-05-31 03:00:00,151399,Подписка ПРО ЛК 151399,5904130450,nan,Подписка ПРО ЛК 151399,40.0,Подписка ПРО,2.7,5.4,2.0


In [37]:
dataset_ready.to_excel('C:/Users/Boris/Downloads/'+ name + '_ready.xlsx', index=False)